In [12]:
#imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression, LassoCV, RidgeCV, LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor

In [31]:
train = pd.read_csv('/Users/Sam/DSI/Projects/project-2-master/datasets/train.csv')
test = pd.read_csv('/Users/Sam/DSI/Projects/project-2-master/datasets/test.csv')


In [33]:
num_feat = list(train._get_numeric_data().drop(['Id', 'PID','SalePrice'], axis=1).columns)

In [34]:
feat = num_feat

In [35]:
for feature in num_feat:
    train[feature] = train[feature].fillna(-998)
    test[feature] = test[feature].fillna(-998)

##  Model

In [39]:
cat_col = list(train.select_dtypes(include=['object']).columns)

In [42]:
for col in cat_col:
    train[col] = train[col].fillna('N/A')
    test[col] = test[col].fillna('N/A')
    train_values = sorted(list(train[col].unique()))
    test_values = sorted(list(test[col].unique()))
    categories = set(train_values + test_values)
    train[col] = pd.Categorical(train[col], categories=categories)
    test[col] = pd.Categorical(test[col], categories=categories)

In [43]:
train_dummies = pd.get_dummies(train[cat_col])
test_dummies = pd.get_dummies(test[cat_col])

In [47]:
X_train = pd.concat([train_dummies, train[num_feat]], axis =1)
X_test = pd.concat([test_dummies, test[num_feat]], axis =1)

In [48]:
params = {}
gs = GridSearchCV(RandomForestRegressor(), param_grid=params)
gs.fit(X_train[num_feat], train['SalePrice'])



/Users/Sam/anaconda/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
/Users/Sam/anaconda/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/Sam/anaconda/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/Sam/anaconda/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/Sam

GridSearchCV(cv='warn', error_score='raise-deprecating',
       estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=None, param_grid={},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [49]:
gs.best_score_

0.8658542513168158

In [50]:
pred = gs.predict(X_test[feat])

In [51]:
test['SalePrice'] = pred

In [52]:
test[['Id', 'SalePrice']].to_csv('submission.csv', index=False)